# Training

## Imports

### Libraries

In [1]:
from contextlib import redirect_stderr
import json
from pathlib import Path
import pickle

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.fixes import loguniform

### Data

In [2]:
train_data = dict()
train_data_path = Path('../data/train_data.pkl')
with train_data_path.open('rb') as file:
    train_data['in'] = pickle.load(file)

test_data_path = Path('../data/test_data.pkl')
with test_data_path.open('rb') as file:
    test_data = pickle.load(file)

groups_path = Path('../data/groups.json')
with groups_path.open('r') as file:
    groups = json.load(file)

## Functions definitions

### Pre-processing functions

As discussed during the exploratory data analysis, there are several pre-processing steps that could be taken. This section defines those steps as functions.

In [3]:
def drop_redundant_columns(df):
    '''
    Drop redundant columns from the ``DataFrame``.

    Returns the ``DataFrame`` with the ``_diff`` and ``_diff_rel`` columns
    removed.
    '''
    redundant_features = (
        df.columns
        .str.extract('(\w+_diff(?:_rel)?(?:__.+)?)')
        .squeeze()
        .dropna()
    )

    df_ = df.drop(redundant_features, axis=1)

    return df_


def drop_duplicate_columns(df, groups=groups):
    '''
    Drop ``DataFrame``'s repeated columns.

    Since the ``DataFrame`` only has duplicated columns in the ``labs``
    category, they can be dropped directly.
    '''
    # Since some of the columns may have been dropped in a previous operation,
    # it is necessary to get the intersection of ``groups['labs']`` with the
    # columns present in ``df`` to avoid a ``KeyError``.
    columns = df.columns.intersection(groups['labs'])

    cols_to_drop = (
        df.loc[:, columns]
        .columns
        .str.extract('(\w+_(?:mean|median|min|diff|diff_rel)(?:__.+)?)')
        .squeeze()
        .dropna()
    )

    df_ = df.drop(cols_to_drop, axis=1)

    return df_


def impute_data(df):
    '''
    Impute missing data.

    Fills ``DataFrame`` by each ``id`` group, using backwards and forwards
    fill, in this order.
    '''
    df_ = (
        df.groupby('id')
        .transform(lambda col: col.bfill().ffill())
    )

    return df_

def one_hot_encode(df):
    '''
    One-hot encode ``age_percentil`` column.

    Returns ``DataFrame`` with the column ``age_percentil`` substituted by a
    set of columns with its one-hot-encoded values.
    '''
    df_ = df.drop(['age_percentil'], axis=1)

    dummies = pd.get_dummies(df['age_percentil'])
    for col in dummies:
        df_.insert(1, col, dummies[col])

    return df_

def reencode_icu(df):
    '''
    Return ``df`` with ``icu`` column reencoded.

    The returned ``DataFrame``'s ``icu`` column is transformed such that
    every row for a given patient is equal to 1 if the patient was
    admitted at any point in time and 0 otherwise.
    '''
    df_ = df.copy()

    df_.loc[:, 'icu'] = (
        df_.loc[:, 'icu']
        .groupby('id')
        .transform('max')
    )

    return df_


def process_rows(df, how='every_row', groups=groups):
    '''
    Process ``DataFrame`` according to discussion in the EDA.
    '''
    df_ = df.copy()

    df_ = reencode_icu(df_)

    if how == 'every_row':
        # Nothing has to be done in this case.
        df_ = df_

    elif how == 'first_window':
        df_ = df_.loc[(slice(None), '0-2'), :]

    elif how == 'aggregate':
        # The features in ``labs`` and ``vitals`` will be aggregated by ``mean``.
        features_to_agg_by_mean = groups['labs'] + groups['vitals']
        # All other features will be aggregated by ``max``.
        features_to_agg_by_max = df_.columns.difference(features_to_agg_by_mean)

        agg_funcs = {
            feature: 'max'
            for feature in features_to_agg_by_max
            if feature in df_.columns
        }

        agg_funcs.update({
            feature: 'mean'
            for feature in features_to_agg_by_mean
            if feature in df_.columns
        })

        df_ = (
            df_.groupby('id')
            .agg(agg_funcs)
            # The method ``loc`` sorts the columns as they were coming in.
            .loc[:, df_.columns]
        )

    elif how == 'pivot':
        # Only ``labs`` and ``vitals`` have different values for the distinct
        # time windows.  The intersection prevents a ``KeyError`` due to
        # dropped columns.
        features_to_pivot = df_.columns.intersection(groups['labs']+groups['vitals'])

        pivoted_df = df_[features_to_pivot].unstack('window')

        # Flattens the ``MultiIndex`` to a single level.
        pivoted_df.columns = pivoted_df.columns.map(
            lambda multiindex_tuple: '__'.join(multiindex_tuple).replace('-', '_')
        )

        # Reconstructs the whole ``DataFrame`` by concatenating with the
        # remaining features aggregated.
        df_ = pd.concat(
            [
                df_[groups['demographics']+groups['comorbidities']].groupby('id').max(),
                pivoted_df,
                df_['icu'].groupby('id').first(),
            ],
            axis=1,
        )
    
    else:
        print('Invalid option passed to "how" parameter.')

    return df_

### Training function

In [4]:
def score_on_first_window(y_true, y_pred):
    '''
    Get score on first window.

    Returns the recall score the model has calculated considering just the first
    time window.
    '''
    # Since there are 5 time windows, take every 5 rows.
    score_first_window = recall_score(y_true[::5], y_pred[::5])

    return score_first_window


def train_models(train_data, models, preprocessings, n_iter=5):
    '''
    Train given models on passed data.

    Constructs a ``Pipeline`` and trains it through a ``RandomizedSearchCV`` for
    every combination of pre-processing and model.

    Returns
    -------
    search_results : ``pandas.DataFrame``
        Models and their results, ordered by performance.
    best_models : ``dict``
        Best ``sklearn`` estimator for each trained model class.
    '''
    search_results = pd.DataFrame()
    best_models = {
        'every_row': dict(),
        'first_window': dict(),
        'aggregate': dict(),
    }

    for model_name, model_dict in models.items():
        for preprocessing in preprocessings:
            X = train_data[preprocessing].drop(['icu'], axis=1)
            y = train_data[preprocessing]['icu']

            pipeline = make_pipeline(
                SimpleImputer(),
                model_dict['model'],
            )

            search = RandomizedSearchCV(
                pipeline,
                param_distributions=model_dict['search_params'],
                n_iter=n_iter,
                random_state=2937103,
            )
            
            # Sometimes sklearn appears to raise a ValueError instead of a
            # warning when an impossible combination of parameters is supplied
            # to RandomizedSearchCV. The try-except block prevents the procedure
            # from being stopped.
            try:
                search.fit(X, y)
            except ValueError as error:
                print(error, file=sys.stderr)
            else:
                results = pd.DataFrame(search.cv_results_)
                results.insert(0, 'pre-processing', preprocessing)
                results.insert(1, 'model', model_name)

                search_results = search_results.append(results, ignore_index=True)

            best_models[preprocessing][model_name] = search.best_estimator_

    search_results = (
        search_results
        .sort_values(by='mean_test_score', ascending=False)
        .reset_index(drop=True)
    )

    return search_results, best_models

## Pre-processing

Before passing the *train_data* to ``scikit-learn``, it has to go through the pre-processing steps that were discussed during the EDA.

In [5]:
train_data['preprocessed'] = (
    train_data['in']
    .pipe(drop_redundant_columns)
    .pipe(drop_duplicate_columns)
    .pipe(impute_data)
    .pipe(one_hot_encode)
    .pipe(reencode_icu)
)

preprocessings = ['every_row', 'first_window', 'aggregate']
for preprocessing in preprocessings:
    train_data[preprocessing] = process_rows(train_data['preprocessed'], how=preprocessing)

The *test_data* also goes through all of the pre-processing, but the rows processing.

In [ ]:
test_data = (
    test_data
    .pipe(drop_redundant_columns)
    .pipe(drop_duplicate_columns)
    .pipe(impute_data)
    .pipe(one_hot_encode)
    .pipe(reencode_icu)
)

## Training

### Models definition

Running a ``RandomizedSearchCV`` requires the definition of a set of parameters to be explored. As many models will be tried, a dictionary containing the model objects and the parameters is defined.

In [7]:
models = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=10000),
        'search_params': {
            'logisticregression__penalty': ['l1', 'l2'],
            'logisticregression__C': loguniform(1e-1, 1e3),
            'logisticregression__solver': ['lbfgs', 'liblinear', 'sag'],
        },
    },
    'DecisionTreeClassifier': {
        'model': DecisionTreeClassifier(),
        'search_params': {
            'decisiontreeclassifier__criterion': ['gini', 'entropy'],
            'decisiontreeclassifier__splitter': ['best', 'random'],
            'decisiontreeclassifier__max_depth': range(2, 10),
        },
    },
    'KNeighborsClassifier': {
        'model': KNeighborsClassifier(),
        'search_params': {
            'kneighborsclassifier__n_neighbors': range(2,20),
            'kneighborsclassifier__weights': ['uniform', 'distance'],
            'kneighborsclassifier__p': [1, 2],
        },
    },
    'LinearSVC': {
        'model': LinearSVC(max_iter=10000),
        'search_params': {
            'linearsvc__penalty': ['l1', 'l2'],
            'linearsvc__loss': ['hinge', 'squared_hinge'],
            'linearsvc__C': loguniform(1e-1, 1e3),
        },
    },
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'search_params': {
            'randomforestclassifier__criterion': ['gini', 'entropy'],
            'randomforestclassifier__n_estimators': [50, 100, 150, 200],
        },
    },
}

### Training

Finally models are created.

In [8]:
# To avoid filling up the screen when parameters are invalid, all warnings are
# redirected to an external file.
sklearn_warnings_path = Path('../sklearn-warnings.txt')
with sklearn_warnings_path.open('w') as file:
    with redirect_stderr(file):

        search_results, best_models = train_models(
            train_data,
            models,
            preprocessings,
            # ['first_window'],
            n_iter=4,
        )

In [ ]:
search_results[['pre-processing', 'model', 'mean_test_score']].iloc[:10, :]

## Testing

It's informative to see a table with a summary of the models' performances on the test set that was held out.

In [10]:
test_X = test_data.drop(['icu'], axis=1)
test_y = test_data['icu']

time_windows = ['0-2', '2-4', '4-6', '6-12', 'above_12']
df = pd.DataFrame()

for preprocessing, model_dict in best_models.items():
    for model_name, model in model_dict.items():
        predictions = pd.Series(
            model.predict(test_X),
            index=test_y.index,
            name='prediction'
        )

        for time_window in time_windows:
            rows_to_consider = (slice(None), time_window) 
            test = test_y[rows_to_consider]
            pred = predictions[rows_to_consider]
            df = (
                df
                .append(
                    {
                        'pre-processing': preprocessing,
                        'model': model_name,
                        'window': time_window,
                        'accuracy': accuracy_score(test, pred),
                        'precision': precision_score(test, pred),
                        'recall': recall_score(test, pred),
                    },
                    ignore_index=True,
                )
            )

df = df.set_index(['model', 'pre-processing', 'window']).unstack(2)

,pre-processing,model,mean_test_score
0,aggregate,RandomForestClassifier,0.842105
1,aggregate,RandomForestClassifier,0.838596
2,aggregate,RandomForestClassifier,0.838596
3,aggregate,RandomForestClassifier,0.835088
4,aggregate,LinearSVC,0.821053
5,aggregate,LogisticRegression,0.817544
6,aggregate,DecisionTreeClassifier,0.814035
7,every_row,RandomForestClassifier,0.808421
8,every_row,RandomForestClassifier,0.807719
9,aggregate,LogisticRegression,0.807018


Looking just at the ``recall`` and sorting by performance on the first time window, it can be seen that models trained using only the first window performed the best. Especially so are the ``RandomForestClassifier`` and the ``LogisticRegression``.

In [11]:
df['recall'].sort_values(by='0-2', ascending=False)

,window,0-2,2-4,4-6,6-12,above_12
model,pre-processing,,,,,
RandomForestClassifier,first_window,0.784314,0.745098,0.784314,0.784314,0.843137
LogisticRegression,first_window,0.745098,0.666667,0.784314,0.882353,0.960784
RandomForestClassifier,every_row,0.745098,0.764706,0.862745,0.882353,0.901961
LinearSVC,first_window,0.725490,0.647059,0.745098,0.862745,0.980392
DecisionTreeClassifier,first_window,0.705882,0.588235,0.686275,0.745098,0.705882
LogisticRegression,every_row,0.686275,0.666667,0.823529,0.862745,0.862745
DecisionTreeClassifier,every_row,0.666667,0.666667,0.745098,0.764706,0.764706
LinearSVC,every_row,0.666667,0.588235,0.803922,0.862745,0.843137
KNeighborsClassifier,first_window,0.529412,0.588235,0.784314,0.784314,0.784314


Taking those two best performing models, a table showing cumulative probability of prediction as a function of time window is created.

In [12]:
def get_performance(preprocessing, model):
    '''
    '''
    predictions = pd.Series(
        best_models[preprocessing][model].predict(test_X),
        name='predictions',
        index=test_y.index,
    )

    admitted_patients = test_y.groupby('id').filter(lambda s: s.max() == 1)

    admitted_patients_predictions = predictions.reindex_like(admitted_patients).to_frame()

    def get_window_of_prediction(group):
        from numpy import nan
        window = group.idxmax()

        try:
            window = window[1]
        except TypeError:
            window = nan

        return window

    windows = (
        admitted_patients_predictions[admitted_patients_predictions == 1]
        .groupby('id')
        .agg(
            window_of_prediction=('predictions', get_window_of_prediction)
        )
        .squeeze()
    )

    cumulative_prob_of_prediction = windows.value_counts(normalize=True, dropna=False).sort_index().cumsum().rename(model)

    return cumulative_prob_of_prediction

Although the ``RandomForestClassifier`` is more capable of getting correct prediction of admission during the first time window, it can be seen that the ``LogisticRegression`` takes over for later time windows.

In [13]:
pd.concat(
    [
        get_performance('first_window', 'RandomForestClassifier'),
        get_performance('first_window', 'LogisticRegression'),
    ],
    axis=1,
).sort_index()

,RandomForestClassifier,LogisticRegression
0-2,0.784314,0.745098
2-4,0.823529,0.784314
4-6,0.941176,0.941176
6-12,NaN,0.960784
above_12,NaN,0.980392
NaN,1.000000,1.000000


## Conclusion

Several different models were trained and evaluated. For each of those models, several distinct pre-processing procedures were attempted, namely, 
- taking only the first time window for each patient,
- aggregating all time windows into one,
- using every row of the dataset as if corresponding to distinct patients.

The performances were evaluated based on the recall, which is the probability of correctly guessing admission.
The two best performing models were a ``RandomForestClassifier`` and a ``LogisticRegression``, both trained on just the data for each patient's first time window.